In [4]:
# pip install selenium

     ---------------------------------------- 5.2/5.2 MB 8.5 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:

import pandas as pd
import numpy as np
from selenium import webdriver
import requests
import bs4
from bs4 import BeautifulSoup
import time

In [5]:
qualifying_results = pd.DataFrame()
for year in list(range(1983,2023)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)

        year_df = pd.concat([year_df, df])

    qualifying_results = pd.concat([qualifying_results, year_df])
    
print(qualifying_results.shape)

(15769, 7)


In [6]:
qualifying_results.head()

,Pos,No,Driver,Car,Time,season,round
0,1,1,Keke Rosberg ROS,Williams Honda,1:34.526,1983,1
1,2,15,Alain Prost PRO,Renault,1:34.672,1983,1
2,3,27,Patrick Tambay TAM,Ferrari,1:34.758,1983,1
3,4,5,Nelson Piquet PIQ,Brabham BMW,1:35.114,1983,1
4,5,35,Derek Warwick WAR,Toleman Hart,1:35.206,1983,1


In [7]:
qualifying_results.tail()

,Pos,No,Driver,Car,Time,season,round
15,16,20,Kevin Magnussen MAG,Haas Ferrari,1:25.834,2022,22
16,17,10,Pierre Gasly GAS,AlphaTauri RBPT,1:25.859,2022,22
17,18,77,Valtteri Bottas BOT,Alfa Romeo Ferrari,1:25.892,2022,22
18,19,23,Alexander Albon ALB,Williams Mercedes,1:26.028,2022,22
19,20,6,Nicholas Latifi LAT,Williams Mercedes,1:26.054,2022,22


In [8]:
qualifying_results.rename(columns = {'Pos': 'grid_position', 'Driver': 'driver_name', 'Car': 'car',
                                     'Time': 'qualifying_time'}, inplace = True)

In [9]:
qualifying_results.drop('No', axis = 1, inplace = True)

In [10]:
qualifying_results.to_csv('qualifying.csv', index = False)